In [28]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import svm
from sklearn.svm import SVC
from nltk import ConfusionMatrix 
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras


In [2]:
uscities_df = pd.read_csv("Clean_data/City_Updated_JS.csv")
pd.set_option("display.max_rows", 100)
uscities_df

city state_id  state_name  population  density
0         New York       NY    New York    18713220    10715
1      Los Angeles       CA  California    12750807     3276
2          Chicago       IL    Illinois     8604203     4574
3            Miami       FL     Florida     6445545     5019
4           Dallas       TX       Texas     5743938     1526
...            ...      ...         ...         ...      ...
28333        Gross       NE    Nebraska           2        6
28334       Lotsee       OK    Oklahoma           2       39
28335    The Ranch       MN   Minnesota           2        2
28336     Shamrock       OK    Oklahoma           2        2
28337       Monowi       NE    Nebraska           1        1

[28338 rows x 5 columns]

In [4]:
income_df = pd.read_csv("Clean_data/income_updated_JS.csv",engine='python')
income_df = income_df.rename(columns={"City": "city", "State_Name": "state_name", "State_ab": "state_id"})
income_df = income_df.groupby(["city", "state_name"], as_index=False).agg({"state_id": "first", "ALand": "mean", "AWater": "mean", "Median": "mean", "Stdev": "mean"})
pd.set_option("display.max_rows", 1000)
income_df


city      state_name state_id         ALand        AWater  \
0          Abbeville         Alabama       AL  8.970592e+07  1.390840e+05   
1          Abbeville       Louisiana       LA  8.357011e+07  1.004953e+08   
2          Abbeville  South Carolina       SC  1.053923e+08  1.623000e+06   
3         Abbotsford       Wisconsin       WI  1.051618e+08  3.842430e+05   
4           Aberdeen        Maryland       MD  2.883998e+07  2.077533e+06   
...              ...             ...      ...           ...           ...   
11222     Zionsville         Indiana       IN  1.141125e+08  9.399300e+04   
11223  Zolfo Springs         Florida       FL  4.522628e+06  0.000000e+00   
11224       Zumbrota       Minnesota       MN  1.012979e+08  9.444000e+04   
11225           Zuni      New Mexico       NM  3.321720e+07  0.000000e+00   
11226         Zwolle       Louisiana       LA  2.779661e+07  1.511233e+06   

             Median         Stdev  
0      25216.000000  39126.000000  
1      25628.333333  21177.666667  
2      32245.000000  41647.333333  
3      44919.000000  48933.000000  
4      80370.000000  64102.000000  
...             ...           ...  
11222  73710.000000  51584.000000  
11223  32041.000000  29600.000000  
11224  45387.000000  44259.500000  
11225  36914.000000  41463.000000  
11226  26065.000000  31901.000000  

[11227 rows x 7 columns]

In [5]:
outcome_df = pd.read_csv("Clean_data/mental_updated.csv")
outcome_df = outcome_df.rename(columns={"State_name": "state_name", "State_ab": "state_id"})
outcome_df

city state_id      state_name  Low_Confidence_Limit  \
0              Abilene       TX           Texas             13.741026   
1                Akron       OH            Ohio             15.933824   
2              Alameda       CA      California              9.211111   
3               Albany       GA         Georgia             14.456364   
4          Albuquerque       NM      New Mexico             12.920863   
5           Alexandria       VA        Virginia              9.555000   
6             Alhambra       CA      California              9.213636   
7                Allen       TX           Texas              9.684615   
8            Allentown       PA    Pennsylvania             16.385714   
9             Amarillo       TX           Texas             13.403509   
10             Anaheim       CA      California             11.780519   
11           Anchorage       AK          Alaska             10.540351   
12           Ann Arbor       MI        Michigan             10.828571   
13             Antioch       CA      California             13.113636   
14        Apple Valley       CA      California             13.421429   
15            Appleton       WI       Wisconsin             10.880769   
16           Arlington       TX           Texas             12.660000   
17   Arlington Heights       IL        Illinois              8.704167   
18              Arvada       CO        Colorado              9.925714   
19           Asheville       NC  North Carolina             11.319355   
20              Athens       GA         Georgia             13.781250   
21             Atlanta       GA         Georgia             11.674242   
22              Auburn       WA      Washington             13.404762   
23             Augusta       GA         Georgia             14.183673   
24              Aurora       CO        Colorado             11.734351   
25              Austin       TX           Texas             10.944000   
26            Avondale       AZ         Arizona             13.091304   
27         Bakersfield       CA      California             13.714815   
28        Baldwin Park       CA      California             11.785000   
29           Baltimore       MD        Maryland             14.707463   
30         Baton Rouge       LA       Louisiana             15.571212   
31             Baytown       TX           Texas             13.435000   
32            Beaumont       TX           Texas             13.747222   
33           Beaverton       OR          Oregon             11.248148   
34            Bellevue       WA      Washington              8.623333   
35          Bellflower       CA      California             12.375000   
36          Bellingham       WA      Washington             12.357895   
37                Bend       OR          Oregon             11.757143   
38            Berkeley       CA      California              9.514286   
39           Bethlehem       PA    Pennsylvania             13.685000   
40            Billings       MT         Montana             11.227586   
41          Birmingham       AL         Alabama             14.928261   
42         Bloomington       IL        Illinois             11.487324   
43          Boca Raton       FL         Florida              9.810345   
44               Boise       ID           Idaho             10.540426   
45         Bolingbrook       IL        Illinois             10.900000   
46              Boston       MA   Massachusetts             12.912000   
47             Boulder       CO        Colorado              9.426667   
48       Boynton Beach       FL         Florida             12.007692   
49          Bridgeport       CT     Connecticut             13.375000   
50            Brockton       MA   Massachusetts             15.047826   
51        Broken Arrow       OK        Oklahoma             12.476923   
52       Brooklyn Park       MN       Minnesota             10.113333   
53         Brownsville       TX           Texas             14.152000   
54 

In [6]:
merged_df = pd.merge(outcome_df, uscities_df, on=['city', 'state_id'], how='left').merge(income_df,on=['city', 'state_id'], how='left')
merged_df = merged_df.rename(columns={"state_name_x": "state_name"})
merged_df = merged_df.drop('state_name_y', axis=1)
merged_df = merged_df.drop(['population'], axis=1)
#pd.set_option("display.max_rows", 500)
merged_df

city state_id      state_name  Low_Confidence_Limit  \
0              Abilene       TX           Texas             13.741026   
1                Akron       OH            Ohio             15.933824   
2              Alameda       CA      California              9.211111   
3               Albany       GA         Georgia             14.456364   
4          Albuquerque       NM      New Mexico             12.920863   
5           Alexandria       VA        Virginia              9.555000   
6             Alhambra       CA      California              9.213636   
7                Allen       TX           Texas              9.684615   
8            Allentown       PA    Pennsylvania             16.385714   
9             Amarillo       TX           Texas             13.403509   
10             Anaheim       CA      California             11.780519   
11           Anchorage       AK          Alaska             10.540351   
12           Ann Arbor       MI        Michigan             10.828571   
13             Antioch       CA      California             13.113636   
14        Apple Valley       CA      California             13.421429   
15            Appleton       WI       Wisconsin             10.880769   
16           Arlington       TX           Texas             12.660000   
17   Arlington Heights       IL        Illinois              8.704167   
18              Arvada       CO        Colorado              9.925714   
19           Asheville       NC  North Carolina             11.319355   
20              Athens       GA         Georgia             13.781250   
21             Atlanta       GA         Georgia             11.674242   
22              Auburn       WA      Washington             13.404762   
23             Augusta       GA         Georgia             14.183673   
24              Aurora       CO        Colorado             11.734351   
25              Austin       TX           Texas             10.944000   
26            Avondale       AZ         Arizona             13.091304   
27         Bakersfield       CA      California             13.714815   
28        Baldwin Park       CA      California             11.785000   
29           Baltimore       MD        Maryland             14.707463   
30         Baton Rouge       LA       Louisiana             15.571212   
31             Baytown       TX           Texas             13.435000   
32            Beaumont       TX           Texas             13.747222   
33           Beaverton       OR          Oregon             11.248148   
34            Bellevue       WA      Washington              8.623333   
35          Bellflower       CA      California             12.375000   
36          Bellingham       WA      Washington             12.357895   
37                Bend       OR          Oregon             11.757143   
38            Berkeley       CA      California              9.514286   
39           Bethlehem       PA    Pennsylvania             13.685000   
40            Billings       MT         Montana             11.227586   
41          Birmingham       AL         Alabama             14.928261   
42         Bloomington       IL        Illinois             11.487324   
43          Boca Raton       FL         Florida              9.810345   
44               Boise       ID           Idaho             10.540426   
45         Bolingbrook       IL        Illinois             10.900000   
46              Boston       MA   Massachusetts             12.912000   
47             Boulder       CO        Colorado              9.426667   
48       Boynton Beach       FL         Florida             12.007692   
49          Bridgeport       CT     Connecticut             13.375000   
50            Brockton       MA   Massachusetts             15.047826   
51        Broken Arrow       OK        Oklahoma             12.476923   
52       Brooklyn Park       MN       Minnesota             10.113333   
53         Brownsville       TX           Texas             14.152000   
54 

In [108]:
#merged_df.loc[pd.isna(merged_df["population"]), :].index
merged_df.isnull().sum(axis = 0)

city                     0
state_id                 0
state_name               0
Low_Confidence_Limit     0
Data_Value               0
High_Confidence_Limit    0
PopulationCount          0
density                  0
state_name               0
ALand                    0
AWater                   0
Median                   0
Stdev                    0
dtype: int64

In [109]:
merged_df["Data_Value"].median()

13.89175824

In [110]:
merged_df["Data_Value"].max()

20.13902439

In [21]:
binary_outcome = pd.cut(merged_df["Data_Value"],bins=[0,13.89175824,21],labels=[0,1])
merged_df = merged_df.drop(["health_bad"],axis=1)
merged_df.insert(5, "health_bad", binary_outcome)
merged_df

city state_id      state_name  Low_Confidence_Limit  \
0              Abilene       TX           Texas             13.741026   
1                Akron       OH            Ohio             15.933824   
2              Alameda       CA      California              9.211111   
3               Albany       GA         Georgia             14.456364   
4          Albuquerque       NM      New Mexico             12.920863   
5           Alexandria       VA        Virginia              9.555000   
6             Alhambra       CA      California              9.213636   
7                Allen       TX           Texas              9.684615   
8            Allentown       PA    Pennsylvania             16.385714   
9             Amarillo       TX           Texas             13.403509   
10             Anaheim       CA      California             11.780519   
11           Anchorage       AK          Alaska             10.540351   
12           Ann Arbor       MI        Michigan             10.828571   
13             Antioch       CA      California             13.113636   
14        Apple Valley       CA      California             13.421429   
15            Appleton       WI       Wisconsin             10.880769   
16           Arlington       TX           Texas             12.660000   
17   Arlington Heights       IL        Illinois              8.704167   
18              Arvada       CO        Colorado              9.925714   
19           Asheville       NC  North Carolina             11.319355   
20              Athens       GA         Georgia             13.781250   
21             Atlanta       GA         Georgia             11.674242   
22              Auburn       WA      Washington             13.404762   
23             Augusta       GA         Georgia             14.183673   
24              Aurora       CO        Colorado             11.734351   
25              Austin       TX           Texas             10.944000   
26            Avondale       AZ         Arizona             13.091304   
27         Bakersfield       CA      California             13.714815   
28        Baldwin Park       CA      California             11.785000   
29           Baltimore       MD        Maryland             14.707463   
30         Baton Rouge       LA       Louisiana             15.571212   
31             Baytown       TX           Texas             13.435000   
32            Beaumont       TX           Texas             13.747222   
33           Beaverton       OR          Oregon             11.248148   
34            Bellevue       WA      Washington              8.623333   
35          Bellflower       CA      California             12.375000   
36          Bellingham       WA      Washington             12.357895   
37                Bend       OR          Oregon             11.757143   
38            Berkeley       CA      California              9.514286   
39           Bethlehem       PA    Pennsylvania             13.685000   
40            Billings       MT         Montana             11.227586   
41          Birmingham       AL         Alabama             14.928261   
42         Bloomington       IL        Illinois             11.487324   
43          Boca Raton       FL         Florida              9.810345   
44               Boise       ID           Idaho             10.540426   
45         Bolingbrook       IL        Illinois             10.900000   
46              Boston       MA   Massachusetts             12.912000   
47             Boulder       CO        Colorado              9.426667   
48       Boynton Beach       FL         Florida             12.007692   
49          Bridgeport       CT     Connecticut             13.375000   
50            Brockton       MA   Massachusetts             15.047826   
51        Broken Arrow       OK        Oklahoma             12.476923   
52       Brooklyn Park       MN       Minnesota             10.113333   
53         Brownsville       TX           Texas             14.152000   
54 

In [112]:
merged_df.to_csv("merged_data.csv", index=False)

In [41]:
feature_cols=['density','ALand','AWater','Median','Stdev']
X=merged_df[feature_cols]
y=merged_df['health_bad'] 
scaler = StandardScaler()
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
# logistic regression
logreg=LogisticRegression()
pipeline = Pipeline([('transformer', scaler), ('estimator', logreg)])
logreg_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, logreg_predicted, output_dict=True)
logreg_result_df = pd.DataFrame(report).transpose()
logreg_result_df = logreg_result_df.rename(columns={"precision": "logreg_precision", "recall": "logreg_recall", "f1-score": "logreg_f1score"})
logreg_result_df = logreg_result_df.drop(columns={'support'})
logreg_result_df = logreg_result_df.drop(index={'macro avg','weighted avg'})
logreg_result_df

logreg_precision  logreg_recall  logreg_f1score
0                 0.826484       0.763713        0.793860
1                 0.780392       0.839662        0.808943
accuracy          0.801688       0.801688        0.801688

In [14]:
svm1 = svm.SVC(kernel='linear')
pipeline = Pipeline([('transformer', scaler), ('estimator', svm1)])
svm1_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, svm1_predicted, output_dict=True)
svm1_result_df = pd.DataFrame(report).transpose()
svm1_result_df = svm1_result_df.rename(columns={"precision": "svm1_precision", "recall": "svm1_recall", "f1-score": "svm1_f1score"})
svm1_result_df = svm1_result_df.drop(columns={'support'})
svm1_result_df = svm1_result_df.drop(index={'macro avg','weighted avg'})
svm1_result_df
#svm2 = svm.SVC(kernel='linear', C=2)
#svm3 = svm.SVC(kernel='linear', C=3)

svm1_precision  svm1_recall  svm1_f1score
0               0.830986     0.746835      0.786667
1               0.770115     0.848101      0.807229
accuracy        0.797468     0.797468      0.797468

In [17]:
svm2 = svm.SVC(kernel='linear',C=2)
pipeline = Pipeline([('transformer', scaler), ('estimator', svm2)])
svm2_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, svm2_predicted, output_dict=True)
svm2_result_df = pd.DataFrame(report).transpose()
svm2_result_df = svm2_result_df.rename(columns={"precision": "svm2_precision", "recall": "svm2_recall", "f1-score": "svm2_f1score"})
svm2_result_df = svm2_result_df.drop(columns={'support'})
svm2_result_df = svm2_result_df.drop(index={'macro avg','weighted avg'})
svm2_result_df

svm2_precision  svm2_recall  svm2_f1score
0               0.831776     0.751055      0.789357
1               0.773077     0.848101      0.808853
accuracy        0.799578     0.799578      0.799578

In [19]:
svm3 = svm.SVC(kernel='linear',C=3)
pipeline = Pipeline([('transformer', scaler), ('estimator', svm3)])
svm3_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, svm3_predicted, output_dict=True)
svm3_result_df = pd.DataFrame(report).transpose()
svm3_result_df = svm3_result_df.rename(columns={"precision": "svm3_precision", "recall": "svm3_recall", "f1-score": "svm3_f1score"})
svm3_result_df = svm3_result_df.drop(columns={'support'})
svm3_result_df = svm3_result_df.drop(index={'macro avg','weighted avg'})
svm3_result_df

svm3_precision  svm3_recall  svm3_f1score
0               0.831776     0.751055      0.789357
1               0.773077     0.848101      0.808853
accuracy        0.799578     0.799578      0.799578

In [21]:
DecisionTree = tree.DecisionTreeClassifier()
pipeline = Pipeline([('transformer', scaler), ('estimator', DecisionTree)])
DecisionTree_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, DecisionTree_predicted, output_dict=True)
DecisionTree_result_df = pd.DataFrame(report).transpose()
DecisionTree_result_df = DecisionTree_result_df.rename(columns={"precision": "DecisionTree_precision", "recall": "DecisionTree_recall", "f1-score": "DecisionTree_f1score"})
DecisionTree_result_df = DecisionTree_result_df.drop(columns={'support'})
DecisionTree_result_df = DecisionTree_result_df.drop(index={'macro avg','weighted avg'})
DecisionTree_result_df

DecisionTree_precision  DecisionTree_recall  DecisionTree_f1score
0                       0.760000             0.721519              0.740260
1                       0.734940             0.772152              0.753086
accuracy                0.746835             0.746835              0.746835

In [22]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
GBoost1 = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=.05,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)
pipeline = Pipeline([('transformer', scaler), ('estimator', GBoost1)])
GBoost1_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, GBoost1_predicted, output_dict=True)
GBoost1_result_df = pd.DataFrame(report).transpose()
GBoost1_result_df = GBoost1_result_df.rename(columns={"precision": "GBoost1_precision", "recall": "GBoost1_recall", "f1-score": "GBoost1_f1score"})
GBoost1_result_df = GBoost1_result_df.drop(columns={'support'})
GBoost1_result_df = GBoost1_result_df.drop(index={'macro avg','weighted avg'})
GBoost1_result_df

GBoost1_precision  GBoost1_recall  GBoost1_f1score
0                  0.834862        0.767932         0.800000
1                  0.785156        0.848101         0.815416
accuracy           0.808017        0.808017         0.808017

In [23]:
GBoost2 = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=.1,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)
pipeline = Pipeline([('transformer', scaler), ('estimator', GBoost2)])
GBoost2_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, GBoost2_predicted, output_dict=True)
GBoost2_result_df = pd.DataFrame(report).transpose()
GBoost2_result_df = GBoost2_result_df.rename(columns={"precision": "GBoost2_precision", "recall": "GBoost2_recall", "f1-score": "GBoost2_f1score"})
GBoost2_result_df = GBoost2_result_df.drop(columns={'support'})
GBoost2_result_df = GBoost2_result_df.drop(index={'macro avg','weighted avg'})
GBoost2_result_df

GBoost2_precision  GBoost2_recall  GBoost2_f1score
0                  0.833333        0.780591         0.806100
1                  0.793651        0.843882         0.817996
accuracy           0.812236        0.812236         0.812236

In [24]:
GBoost3 = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=.25,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)
pipeline = Pipeline([('transformer', scaler), ('estimator', GBoost3)])
GBoost3_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, GBoost3_predicted, output_dict=True)
GBoost3_result_df = pd.DataFrame(report).transpose()
GBoost3_result_df = GBoost3_result_df.rename(columns={"precision": "GBoost3_precision", "recall": "GBoost3_recall", "f1-score": "GBoost3_f1score"})
GBoost3_result_df = GBoost3_result_df.drop(columns={'support'})
GBoost3_result_df = GBoost3_result_df.drop(index={'macro avg','weighted avg'})
GBoost3_result_df

GBoost3_precision  GBoost3_recall  GBoost3_f1score
0                  0.811659        0.763713         0.786957
1                  0.776892        0.822785         0.799180
accuracy           0.793249        0.793249         0.793249

In [25]:
GBoost4 = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=.5,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)
pipeline = Pipeline([('transformer', scaler), ('estimator', GBoost4)])
GBoost4_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, GBoost4_predicted, output_dict=True)
GBoost4_result_df = pd.DataFrame(report).transpose()
GBoost4_result_df = GBoost4_result_df.rename(columns={"precision": "GBoost4_precision", "recall": "GBoost4_recall", "f1-score": "GBoost4_f1score"})
GBoost4_result_df = GBoost4_result_df.drop(columns={'support'})
GBoost4_result_df = GBoost4_result_df.drop(index={'macro avg','weighted avg'})
GBoost4_result_df

GBoost4_precision  GBoost4_recall  GBoost4_f1score
0                  0.791111        0.751055         0.770563
1                  0.763052        0.801688         0.781893
accuracy           0.776371        0.776371         0.776371

In [26]:
GBoost5 = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=.75,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)
pipeline = Pipeline([('transformer', scaler), ('estimator', GBoost5)])
GBoost5_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, GBoost5_predicted, output_dict=True)
GBoost5_result_df = pd.DataFrame(report).transpose()
GBoost5_result_df = GBoost5_result_df.rename(columns={"precision": "GBoost5_precision", "recall": "GBoost5_recall", "f1-score": "GBoost5_f1score"})
GBoost5_result_df = GBoost5_result_df.drop(columns={'support'})
GBoost5_result_df = GBoost5_result_df.drop(index={'macro avg','weighted avg'})
GBoost5_result_df

GBoost5_precision  GBoost5_recall  GBoost5_f1score
0                  0.781513        0.784810         0.783158
1                  0.783898        0.780591         0.782241
accuracy           0.782700        0.782700         0.782700

In [27]:
GBoost6 = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=1,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)
pipeline = Pipeline([('transformer', scaler), ('estimator', GBoost6)])
GBoost6_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, GBoost6_predicted, output_dict=True)
GBoost6_result_df = pd.DataFrame(report).transpose()
GBoost6_result_df = GBoost6_result_df.rename(columns={"precision": "GBoost6_precision", "recall": "GBoost6_recall", "f1-score": "GBoost6_f1score"})
GBoost6_result_df = GBoost6_result_df.drop(columns={'support'})
GBoost6_result_df = GBoost6_result_df.drop(index={'macro avg','weighted avg'})
GBoost6_result_df

GBoost6_precision  GBoost6_recall  GBoost6_f1score
0                  0.777293        0.751055         0.763948
1                  0.759184        0.784810         0.771784
accuracy           0.767932        0.767932         0.767932

In [28]:
BalRandomForest = BalancedRandomForestClassifier(n_estimators=100,random_state=1)
pipeline = Pipeline([('transformer', scaler), ('estimator', BalRandomForest)])
BalRandomForest_predicted = cross_val_predict(pipeline, X, y, cv=10)
report = metrics.classification_report(y, BalRandomForest_predicted, output_dict=True)
BalRandomForest_result_df = pd.DataFrame(report).transpose()
BalRandomForest_result_df = BalRandomForest_result_df.rename(columns={"precision": "BalRandomForest_precision", "recall": "BalRandomForest_recall", "f1-score": "BalRandomForest_f1score"})
BalRandomForest_result_df = BalRandomForest_result_df.drop(columns={'support'})
BalRandomForest_result_df = BalRandomForest_result_df.drop(index={'macro avg','weighted avg'})
BalRandomForest_result_df

BalRandomForest_precision  BalRandomForest_recall  \
0                          0.815789                0.784810   
1                          0.792683                0.822785   
accuracy                   0.803797                0.803797   

          BalRandomForest_f1score  
0                        0.800000  
1                        0.807453  
accuracy                 0.803797

In [31]:
from sklearn.model_selection import cross_validate
output = cross_validate(BalRandomForest, X, y, cv=10, scoring = 'accuracy', return_estimator =True)
for idx,estimator in enumerate(output['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                       index = X.columns,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)

Features sorted by their score for estimator 0:
         importance
Stdev      0.409655
Median     0.272844
ALand      0.110994
density    0.105730
AWater     0.100778
Features sorted by their score for estimator 1:
         importance
Stdev      0.421303
Median     0.261826
AWater     0.108388
ALand      0.107259
density    0.101223
Features sorted by their score for estimator 2:
         importance
Stdev      0.404720
Median     0.275896
density    0.109727
AWater     0.104883
ALand      0.104774
Features sorted by their score for estimator 3:
         importance
Stdev      0.410839
Median     0.256941
ALand      0.115030
density    0.112128
AWater     0.105062
Features sorted by their score for estimator 4:
         importance
Stdev      0.410059
Median     0.271341
ALand      0.111760
AWater     0.105195
density    0.101644
Features sorted by their score for estimator 5:
         importance
Stdev      0.397207
Median     0.269409
AWater     0.111381
ALand      0.111186
density    0

In [70]:
#Custom metric functions
def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def custom_recall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (Positives+K.epsilon())    
    return recall 

def custom_precision(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))    
    precision = TP / (Pred_Positives+K.epsilon())
    return precision 

In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K

from sklearn.metrics import f1_score, make_scorer, confusion_matrix, accuracy_score, precision_score, recall_score, precision_recall_curve
#import numpy as np
scaler = StandardScaler()

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)
    
# K-fold Cross Validation model evaluation
fold_no = 1
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []
    
for train_index, test_index in kfold.split(X, y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Define the model architecture
    model = tf.keras.models.Sequential()
    # Fit the StandardScaler
    X_scaler = scaler.fit(x_train)
    # Scale the data
    X_train_scaled = X_scaler.transform(x_train)
    X_test_scaled = X_scaler.transform(x_test)

    # First hidden layer
    model.add(tf.keras.layers.Dense(units=3, input_dim=5, activation="relu"))
    # Second hidden layer
    #model.add(tf.keras.layers.Dense(units=2, activation="relu"))
    
    # Output layer
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    # Compile the model
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[custom_precision, custom_recall, custom_f1,"accuracy"])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(X_train_scaled, y_train,
              batch_size=10,
              epochs=50,
              verbose=0)

    # Generate generalization metrics
    scores = model.evaluate(X_test_scaled, y_test, verbose=0)
    #print(scores)
    #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%; {model.metrics_names[2]} of {scores[2]*100}%; {model.metrics_names[3]} of {scores[3]*100}%; {model.metrics_names[4]} of {scores[4]*100}%')

    loss_per_fold.append(scores[0])
    precision_per_fold.append(scores[1] * 100)
    recall_per_fold.append(scores[2] * 100)
    f1_per_fold.append(scores[3] * 100)  
    acc_per_fold.append(scores[4] * 100)
        
    y_test = model.predict(X_test_scaled)
    y_test_pred_cat = (np.asarray(y_test)).round()
    
    ### (3) Get performance metrics after each fold
    #f1, precision, recall = f1_score(y_test, y_test_pred_cat), precision_score(y_test, y_test_pred_cat), recall_score(y_test, y_test_pred_cat)
    
    print(f'Fold {fold_no} Loss: %f' %scores[0])
    #precision_per_fold.append(precision * 100)
    
    print(f'Fold {fold_no} Precision: %f' %scores[1])
    #precision_per_fold.append(precision * 100)

    print(f'Fold {fold_no} Recall: %f' %scores[2])
    #recall_per_fold.append(recall * 100)

    print(f'Fold {fold_no} F1 score: %f' %scores[3])
    #f1_per_fold.append(f1 * 100)
    
    print(f'Fold {fold_no} Accuracy: %f' %scores[4])
 
    # Increase fold number
    fold_no = fold_no + 1
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print('------------------------------------------------------------------------')   



------------------------------------------------------------------------
Training for fold 1 ...
Fold 1 Loss: 0.292537
Fold 1 Precision: 0.900000
Fold 1 Recall: 0.944444
Fold 1 F1 score: 0.915033
Fold 1 Accuracy: 0.895833
------------------------------------------------------------------------
Training for fold 2 ...
Fold 2 Loss: 0.444419
Fold 2 Precision: 0.752381
Fold 2 Recall: 0.837500
Fold 2 F1 score: 0.784946
Fold 2 Accuracy: 0.854167
------------------------------------------------------------------------
Training for fold 3 ...
Fold 3 Loss: 0.483457
Fold 3 Precision: 0.692857
Fold 3 Recall: 0.593750
Fold 3 F1 score: 0.639394
Fold 3 Accuracy: 0.729167
------------------------------------------------------------------------
Training for fold 4 ...
Fold 4 Loss: 0.523285
Fold 4 Precision: 0.733333
Fold 4 Recall: 0.911111
Fold 4 F1 score: 0.809941
Fold 4 Accuracy: 0.770833
------------------------------------------------------------------------
Training for fold 5 ...
Fold 5 Loss: 0.

In [73]:
scaler = StandardScaler()

# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)
    
# K-fold Cross Validation model evaluation
fold_no = 1
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
precision_per_fold = []
recall_per_fold = []
f1_per_fold = []


for train_index, test_index in kfold.split(X, y):
    x_train, x_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Define the model architecture
    model = tf.keras.models.Sequential()
    # Fit the StandardScaler
    X_scaler = scaler.fit(x_train)
    # Scale the data
    X_train_scaled = X_scaler.transform(x_train)
    X_test_scaled = X_scaler.transform(x_test)

    # First hidden layer
    model.add(tf.keras.layers.Dense(units=50, input_dim=5, activation="relu"))
    # Second hidden layer
    model.add(tf.keras.layers.Dense(units=20, activation="relu"))
    # Third hidden layer
    #model.add(tf.keras.layers.Dense(units=2, activation="relu"))
    # Output layer
    model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    # Compile the model
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[custom_precision, custom_recall, custom_f1,"accuracy"])


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(X_train_scaled, y_train,
              batch_size=10,
              epochs=50,
              verbose=0)

    # Generate generalization metrics
    scores = model.evaluate(X_test_scaled, y_test, verbose=1)
    #print(scores)
    #print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%; {model.metrics_names[2]} of {scores[2]*100}%; {model.metrics_names[3]} of {scores[3]*100}%; {model.metrics_names[4]} of {scores[4]*100}%')


    loss_per_fold.append(scores[0])
    precision_per_fold.append(scores[1] * 100)
    recall_per_fold.append(scores[2] * 100)
    f1_per_fold.append(scores[3] * 100)  
    acc_per_fold.append(scores[4] * 100)
        
    y_test = model.predict(X_test_scaled)
    y_test_pred_cat = (np.asarray(y_test)).round()
    
    ### (3) Get performance metrics after each fold
    #f1, precision, recall = f1_score(y_test, y_test_pred_cat), precision_score(y_test, y_test_pred_cat), recall_score(y_test, y_test_pred_cat)
    
    
    print(f'Fold {fold_no} Loss: %f' %scores[0])
    #precision_per_fold.append(precision * 100)
    
    print(f'Fold {fold_no} Precision: %f' %scores[1])
    #precision_per_fold.append(precision * 100)

    print(f'Fold {fold_no} Recall: %f' %scores[2])
    #recall_per_fold.append(recall * 100)

    print(f'Fold {fold_no} F1 score: %f' %scores[3])
    #f1_per_fold.append(f1 * 100)
    
    print(f'Fold {fold_no} Accuracy: %f' %scores[4])
 
    # Increase fold number
    fold_no = fold_no + 1
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> Precision: {np.mean(precision_per_fold)} (+- {np.std(precision_per_fold)})')
print(f'> Recall: {np.mean(recall_per_fold)} (+- {np.std(recall_per_fold)})')
print(f'> F1: {np.mean(f1_per_fold)} (+- {np.std(f1_per_fold)})')
print('------------------------------------------------------------------------')   



------------------------------------------------------------------------
Training for fold 1 ...
2/2 [==============================] - 0s 2ms/step - loss: 0.4492 - custom_precision: 0.5636 - custom_recall: 0.7500 - custom_f1: 0.6433 - accuracy: 0.7708
Fold 1 Loss: 0.449168
Fold 1 Precision: 0.563636
Fold 1 Recall: 0.750000
Fold 1 F1 score: 0.643275
Fold 1 Accuracy: 0.770833
------------------------------------------------------------------------
Training for fold 2 ...
2/2 [==============================] - 0s 2ms/step - loss: 0.4576 - custom_precision: 0.8233 - custom_recall: 0.8697 - custom_f1: 0.8452 - accuracy: 0.8333
Fold 2 Loss: 0.457630
Fold 2 Precision: 0.823308
Fold 2 Recall: 0.869748
Fold 2 F1 score: 0.845238
Fold 2 Accuracy: 0.833333
------------------------------------------------------------------------
Training for fold 3 ...
2/2 [==============================] - 0s 2ms/step - loss: 0.3765 - custom_precision: 0.8929 - custom_recall: 0.8929 - custom_f1: 0.8929 - accuracy